# Pynamodb Attribute

In [30]:
import typing as T
from datetime import datetime, timezone

from pynamodb.models import (
    Model,
    PAY_PER_REQUEST_BILLING_MODE,
)
from pynamodb.connection import Connection
from pynamodb.attributes import (
    UnicodeAttribute,
    UTCDateTimeAttribute,
    MapAttribute,
    DynamicMapAttribute,
    JSONAttribute,
)

def utc_now() -> datetime:
    return datetime.utcnow().replace(tzinfo=timezone.utc)

## MapAttribute

Reference:

- https://pynamodb.readthedocs.io/en/latest/attributes.html#map-attributes

In [31]:
class Person(MapAttribute):
    firstname = UnicodeAttribute()
    lastname = UnicodeAttribute()
    birth_time = UTCDateTimeAttribute(default=utc_now)


class DynamodbItem(Model):
    class Meta:
        table_name = "pynamodb-attribute-test"
        region = "us-east-1"
        billing_mode = PAY_PER_REQUEST_BILLING_MODE

    id: T.Union[str, UnicodeAttribute] = UnicodeAttribute(hash_key=True)
    person: T.Optional[Person] = Person(default=None, null=True)

connection = Connection()

In [32]:
DynamodbItem.create_table(wait=True)

In [33]:
item = DynamodbItem(id="id-1")
item.save()

{'ConsumedCapacity': {'CapacityUnits': 1.0,
  'TableName': 'pynamodb-attribute-test'}}

In [34]:
item = DynamodbItem.get("id-1")
print(f"item.attribute_values = {item.attribute_values}")

item.attribute_values = {'id': 'id-1'}


In [35]:
item = DynamodbItem(
    id="id-2",
    person=Person(
        firstname="John",
        lastname="Doe",
    ),
)
item.save()

{'ConsumedCapacity': {'CapacityUnits': 1.0,
  'TableName': 'pynamodb-attribute-test'}}

In [36]:
item = DynamodbItem.get("id-2")
print(f"item.attribute_values = {item.attribute_values}")
print(f"item.person.attribute_values = {item.person.attribute_values}")
print(f"item.person.firstname = {item.person.firstname}")
print(f"item.person.lastname = {item.person.lastname}")
print(f"item.person.birth_time = {item.person.birth_time}")

item.attribute_values = {'id': 'id-2', 'person': <__main__.Person object at 0x107ca0940>}
item.person.attribute_values = {'birth_time': datetime.datetime(2022, 12, 25, 13, 26, 30, 293899, tzinfo=datetime.timezone.utc), 'firstname': 'John', 'lastname': 'Doe'}
item.person.firstname = John
item.person.lastname = Doe
item.person.birth_time = 2022-12-25 13:26:30.293899+00:00


In [37]:
DynamodbItem.delete_table()

{'TableDescription': {'BillingModeSummary': {'BillingMode': 'PAY_PER_REQUEST',
   'LastUpdateToPayPerRequestDateTime': 1671974610.319},
  'ItemCount': 0,
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'TableArn': 'arn:aws:dynamodb:us-east-1:669508176277:table/pynamodb-attribute-test',
  'TableId': '334b34f1-e243-4c05-baae-24c239da0132',
  'TableName': 'pynamodb-attribute-test',
  'TableSizeBytes': 0,
  'TableStatus': 'DELETING',
  'TableThroughputModeSummary': {'LastUpdateToPayPerRequestDateTime': 1671974610.319,
   'TableThroughputMode': 'PAY_PER_REQUEST'}}}

## DynamicMapAttribute

Reference:

- https://pynamodb.readthedocs.io/en/latest/attributes.html#map-attributes

In [42]:
class Person(DynamicMapAttribute):
    firstname = UnicodeAttribute()
    lastname = UnicodeAttribute()


class DynamodbItem(Model):
    class Meta:
        table_name = "pynamodb-attribute-test"
        region = "us-east-1"
        billing_mode = PAY_PER_REQUEST_BILLING_MODE

    id: T.Union[str, UnicodeAttribute] = UnicodeAttribute(hash_key=True)
    person: T.Optional[Person] = Person(default=None, null=True)

connection = Connection()

In [46]:
DynamodbItem.create_table(wait=True)
print("Table created")

Table created


In [52]:
item = DynamodbItem(id="id-1")
item.save()

{'ConsumedCapacity': {'CapacityUnits': 1.0,
  'TableName': 'pynamodb-attribute-test'}}

In [53]:
item = DynamodbItem.get("id-1")
print(f"item.attribute_values = {item.attribute_values}")

item.attribute_values = {'id': 'id-1'}


In [50]:
item = DynamodbItem(
    id="id-2",
    person=Person(
        firstname="John",
        lastname="Doe",
        email="john.doe@email.com", # additional attribute has to be str / binary / int / float type
    ),
)
item.save()

{'ConsumedCapacity': {'CapacityUnits': 1.0,
  'TableName': 'pynamodb-attribute-test'}}

In [51]:
item = DynamodbItem.get("id-2")
print(f"item.attribute_values = {item.attribute_values}")
print(f"item.person.attribute_values = {item.person.attribute_values}")
print(f"item.person.firstname = {item.person.firstname}")
print(f"item.person.lastname = {item.person.lastname}")
print(f"item.person.birth_time = {item.person.email}")

item.attribute_values = {'id': 'id-2', 'person': <__main__.Person object at 0x10bc62be0>}
item.person.attribute_values = {'attribute_values': {}, 'firstname': 'John', 'lastname': 'Doe', 'email': 'john.doe@email.com'}
item.person.firstname = John
item.person.lastname = Doe
item.person.birth_time = john.doe@email.com


In [54]:
DynamodbItem.delete_table()

{'TableDescription': {'BillingModeSummary': {'BillingMode': 'PAY_PER_REQUEST',
   'LastUpdateToPayPerRequestDateTime': 1671974894.754},
  'ItemCount': 0,
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'TableArn': 'arn:aws:dynamodb:us-east-1:669508176277:table/pynamodb-attribute-test',
  'TableId': '80bdeffc-1a8c-4c07-b327-18a3845ea6bd',
  'TableName': 'pynamodb-attribute-test',
  'TableSizeBytes': 0,
  'TableStatus': 'DELETING',
  'TableThroughputModeSummary': {'LastUpdateToPayPerRequestDateTime': 1671974894.754,
   'TableThroughputMode': 'PAY_PER_REQUEST'}}}